In [1]:
import os, pygame, random, math
import numpy as np
import matplotlib.pyplot as plt

pygame 2.4.0 (SDL 2.26.4, Python 3.10.11)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [532]:
colours = {"cyan": (0, 255, 255, 255),
           "blue": (0, 0, 255, 255),
           "orange": (255, 165, 0, 255),
           "yellow": (255, 255, 0, 255),
           "green": (0, 255, 0, 255),
           "purple": (160, 32, 240, 255),
           "red": (255, 0, 0, 255),
           "green2": (165, 255, 81),
           "red2": (255, 76, 76),
           "grey": (140, 140, 140),
           "black": (0, 0, 0),
           "white": (255, 255, 255)}

# Markov Junior

Ordered list of rewrite rules: search over rules in order, if there are matches for the rule select one at random to replace with the rule, else move to the next rule, repeating until no matches are found for any rules.

### Rewrite rules

ABC = XYZ : match instances of A-B-C in grid, replace with X-Y-Z

# Implementation:

### Setup:

Take in board size, cellsize parameters. Stores rules. Eventually could add a graphical way to select initial board.

### Apply Rule:

Given a single rule: search over board to find all matches. If found, randomly change one, else return False.

### General algorithm

Loop over rules, applying them until one changes, end game if none found.

In [533]:
class MarkovJunior():
    def __init__(self, i, j, cellsize):
        os.environ['SDL_VIDEO_CENTERED'] = '1'
        # keep running
        self.game = True
        # store values
        self.i_count = i
        self.j_count = j
        self.cell_size = cellsize
        # setup pygame
        pygame.init()
        self.window = pygame.display.set_mode((self.j_count * self.cell_size, self.i_count * self.cell_size))
        pygame.display.set_caption("MarkovJunior")
        self.clock = pygame.time.Clock()
        # initialise values 
        self.grid = np.empty((self.i_count, self.j_count), dtype=str)
        # colour codes
        self.ccodes = {"W": (255, 255, 255), "B": (0, 0, 0), "R": (255, 0, 0, 255), "G": (0, 255, 0, 255), "O": (255, 165, 0, 255)}
        # rules 
        self.rules = []

    def setup(self):
        """Setup rules and grid."""
        # define rules
        self.rules = [
            ["RBB", "WWR"],
            ["RBW", "GBO"],
            ["GWW", "BBG"],
            ["GWO", "BBR"]
            #["RWB", "BRW"],
            #["RWWB", "BRWW"],
            #["RB", "BR"]
        ]
        # intitialise grid
        for i in range(self.i_count):
            for j in range(self.j_count):
                self.grid[i, j] = "B"
                #if np.random.uniform(0,1) < 0.2:
                #    self.grid[i, j] = "W"
                #if np.random.uniform(0,1) < 0.025:
                #    self.grid[i, j] = "R"
        self.grid[self.i_count // 2, self.j_count // 2] = "R"

    def neighbour_indices(self, i, j, n):
        """
        Return list of: lists of indices n-steps into cardinal directions about (i,j).
        """
        neighbours = []
        # search over directions
        for dx, dy in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
            steps = []
            valid = True
            for k in range(n):
                x = i + k * dx
                y = j + k * dy
                if x < 0 or y < 0 or x >= self.i_count or y >= self.j_count:
                    # out of bounds of grid
                    valid = False
                else:
                    # within bounds of grid
                    steps.append((x, y))
            # valid: no out of bounds indices
            if valid:
                neighbours.append(steps)
        return neighbours

    def apply_rule(self, input, output):
        """Find all instances of rule, randomly select one to replace, or return False."""

        # store match indices: will be list of: lists of indices for each match
        matches = []

        # loop over grid
        for i in range(self.i_count):
            for j in range(self.j_count):
                # entry matches first letter of rule
                if self.grid[i,j] == input[0]:
                    # 1 letter: done
                    if len(input) == 1:
                        # record index
                        matches.append([(i,j)])
                    # more than 1 letter
                    else:
                        # find length
                        n = len(input)
                        # search n-step neighbours
                        nbs = self.neighbour_indices(i, j, n)
                        for steps in nbs:
                            # check if all n entries match rule
                            matching = True
                            for k, (x, y) in enumerate(steps):
                                if not (self.grid[x, y] == input[k]):
                                    matching = False
                            # valid matching
                            if matching:
                                matches.append(steps)

        # no matches found: return False
        if not matches:
            return False
        # otherwise: choose at random from matches
        else:
            indices = random.choice(matches)
            # loop over positions, applying rule
            for k, (i, j) in enumerate(indices):
                # replace by rule
                self.grid[i, j] = output[k]
        # return true if matched and replaced
        return True
    
    def update(self):
        """Main algorithm update step."""
        applied = False
        # loop over rules in order
        for rule in self.rules:
            # apply
            applied = self.apply_rule(rule[0], rule[1])
            # stop if applied
            if applied:
                break
        # if applied False at end: all rules applied found no matches: end
        if not applied:
            self.game = False

    def render(self):
        """Draw the current grid to the screen, colouring according to entries."""
        # define square function
        def square(colour, i, j):
            """Draw square at (i,j) of side length self.cellsize and colour."""
            pygame.draw.rect(self.window, colour, pygame.Rect(
                        j * self.cell_size,
                        i * self.cell_size,
                        self.cell_size,
                        self.cell_size))
        # draw all grid cells
        for i in range(self.i_count):
            for j in range(self.j_count):
                colour = self.ccodes[self.grid[i,j]]
                square(colour, i, j)
        # update screen
        pygame.display.update()

    def user_input(self):
        """Listen for user inputs."""
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                self.game = False
                break
            # listen for key presses
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    self.game = False
                    break

    def run(self):
        """Run main algorithm loop."""
        # call setup
        self.setup()
        # render initial grid
        self.render()

        # loop while running
        while self.game:
            # set tick speed
            self.clock.tick(60)
            # listen for inputs
            self.user_input()
            # update
            self.update()
            # render changes
            self.render()
        
        # no longer running: listen for quit
        self.game = True
        while self.game:
            self.user_input()
        # exit
        pygame.quit()

### Subclasses

Create subclasses for different programmes (sets of rules / grid setups)

In [534]:
class LoopErasedRandomWalk(MarkovJunior):
    def __init__(self, i, j, cellsize):
        super().__init__(i, j, cellsize)

    def setup(self):
        """Setup rules and grid."""
        # define rules
        self.rules = [
            ["RBB", "WWR"],
            ["RBW", "GBO"],
            ["GWW", "BBG"],
            ["GWO", "BBR"]
        ]

        # intitialise grid
        for i in range(self.i_count):
            for j in range(self.j_count):
                # all black
                self.grid[i, j] = "B"
        # starting point in centre
        self.grid[self.i_count // 2, self.j_count // 2] = "R"

In [536]:
rw = LoopErasedRandomWalk(20, 20, 30)
rw.run()

In [537]:
class Sokobhan(MarkovJunior):
    def __init__(self, i, j, cellsize):
        super().__init__(i, j, cellsize)

    def setup(self):
        """Setup rules and grid."""
        # define rules
        self.rules = [
            ["RWB", "BRW"],
            ["RWWB", "BRWW"],
            ["RB", "BR"]
        ]
        # intitialise grid
        for i in range(self.i_count):
            for j in range(self.j_count):
                # default black
                self.grid[i, j] = "B"
                # chance of blocks
                if np.random.uniform(0,1) < 0.2:
                    self.grid[i, j] = "W"
                # chance of agents
                if np.random.uniform(0,1) < 0.025:
                    self.grid[i, j] = "R"
        # agent in middle
        self.grid[self.i_count // 2, self.j_count // 2] = "R"

In [538]:
sb = Sokobhan(20, 20, 30)
sb.run()

# Change architecture:

Implement nodes: collections of rules, with specifications on how to apply them.

Run the whole program in a loop, rendering to the screen at chosen update points

In [539]:
class MarkovJuniorNodes():
    def __init__(self, i, j, cellsize):
        os.environ['SDL_VIDEO_CENTERED'] = '1'
        # keep running
        self.game = True
        # store values
        self.i_count = i
        self.j_count = j
        self.cell_size = cellsize
        # setup pygame
        pygame.init()
        self.window = pygame.display.set_mode((self.j_count * self.cell_size, self.i_count * self.cell_size))
        pygame.display.set_caption("MarkovJunior")
        self.clock = pygame.time.Clock()
        # initialise values 
        self.grid = np.empty((self.i_count, self.j_count), dtype=str)
        # colour codes
        self.ccodes = {"W": (255, 255, 255), "B": (0, 0, 0), "R": (255, 0, 0, 255), "G": (0, 255, 0, 255), "O": (255, 165, 0, 255),
                       "C": (0, 255, 255), "L": (165, 255, 81)}
        # rules 
        self.rules = []
        # program
        self.program = []

    def setup(self):
        """Define rules, nodes and grid."""
        # define program using nodes of rules
        self.program = [
            {"limit": [10, ["B", "W"]]},
            {"limit": [10, ["B", "R"]]},
            {"Sequential": [["WB", "WW"], ["RB", "RR"]]},
            {"Markov": [["RW", "CC"]]},
            {"Markov": [["W", "B"], ["R", "B"]]},
            {"Markov": [["CB", "CL"]]},
            {"limit": [10, ["B", "G"]]},
            {"Sequential": [["GB", "GG"], ["LB", "LL"]]}
        ]
        # optional cleaning
        clean = [
            {"Markov": [["CLC", "CCC"], ["CLLC", "CCCC"]]}
        ]
        # self.program += clean

        # intitialise grid
        for i in range(self.i_count):
            for j in range(self.j_count):
                self.grid[i, j] = "B"

    def neighbour_indices(self, i, j, n):
        """
        Return list of: lists of indices n-steps into cardinal directions about (i,j).
        """
        neighbours = []
        # search over directions
        for dx, dy in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
            steps = []
            valid = True
            for k in range(n):
                x = i + k * dx
                y = j + k * dy
                if x < 0 or y < 0 or x >= self.i_count or y >= self.j_count:
                    # out of bounds of grid
                    valid = False
                else:
                    # within bounds of grid
                    steps.append((x, y))
            # valid: no out of bounds indices
            if valid:
                neighbours.append(steps)
        return neighbours

    def apply_rule(self, input, output):
        """Find all instances of rule, randomly select one to replace, or return False."""

        # store match indices: will be list of: lists of indices for each match
        matches = []

        # loop over grid
        for i in range(self.i_count):
            for j in range(self.j_count):
                # entry matches first letter of rule
                if self.grid[i,j] == input[0]:
                    # 1 letter: done
                    if len(input) == 1:
                        # record index
                        matches.append([(i,j)])
                    # more than 1 letter
                    else:
                        # find length
                        n = len(input)
                        # search n-step neighbours
                        nbs = self.neighbour_indices(i, j, n)
                        for steps in nbs:
                            # check if all n entries match rule
                            matching = True
                            for k, (x, y) in enumerate(steps):
                                if not (self.grid[x, y] == input[k]):
                                    matching = False
                            # valid matching
                            if matching:
                                matches.append(steps)

        # no matches found: return False
        if not matches:
            return False
        # otherwise: choose at random from matches
        else:
            indices = random.choice(matches)
            # loop over positions, applying rule
            for k, (i, j) in enumerate(indices):
                # replace by rule
                self.grid[i, j] = output[k]
        # return true if matched and replaced
        return True
    
    def program_loop(self):
        """Implement program"""
        # loop over nodes in order
        for node in self.program:
            for x, y in node.items():
                node_type = x
                rules = y
            # markov node
            if node_type == "Markov":
                # loop: appling first match and resetting until no matches found
                # while rules being applied
                applied = True
                while applied:
                    applied = False
                    # loop over rules
                    for rule in rules:
                        # apply each in turn
                        applied = self.apply_rule(rule[0], rule[1])
                        # match found: break current loop
                        if applied:
                            # render 
                            self.render()
                            break
                # once no matches: end node call
            elif node_type == "Sequential":
                # loop: attempt to apply each node, until full pass with no matches
                applied = True
                while applied:
                    applied = False
                    # loop over rules
                    for rule in rules:
                        # apply each in turn
                        match = self.apply_rule(rule[0], rule[1])
                        # match found: turn applied true
                        if match:
                            applied = True
                            # render
                            self.render()
                # once no matches in a pass: end node call

            elif node_type == "limit":
                # limit node
                limit = rules[0]
                rule = rules[1]
                # (attempt to) apply rule 'limit' times
                for i in range(limit):
                    applied = self.apply_rule(rule[0], rule[1])
                    if applied:
                        self.render()

    def render(self):
        """Draw the current grid to the screen, colouring according to entries."""
        # define square function
        def square(colour, i, j):
            """Draw square at (i,j) of side length self.cellsize and colour."""
            pygame.draw.rect(self.window, colour, pygame.Rect(
                        j * self.cell_size,
                        i * self.cell_size,
                        self.cell_size,
                        self.cell_size))
        # draw all grid cells
        for i in range(self.i_count):
            for j in range(self.j_count):
                colour = self.ccodes[self.grid[i,j]]
                square(colour, i, j)
        # update screen
        pygame.display.update()

    def user_input(self):
        """Listen for user inputs."""
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
            # listen for key presses
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    pygame.quit()

    def run(self):
        """Run main algorithm loop."""
        # call setup
        self.setup()
        # render initial grid
        self.render()

        # run program
        self.program_loop()
        
        # no longer running: listen for quit
        while True:
            self.user_input()

In [540]:
pygame.quit()

In [541]:
mj = MarkovJuniorNodes(20, 50, 25)
mj.run()

error: video system not initialized

# Implement classes

Create classes for rules and nodes, adding features:

Rules have input and output strings (their re-write rules), as well as an option for a 'field': a proability distribution over the grid that can be used to give non-uniform weights to matches. A method can be called to compute the field for the current state e.g. weighting grid squares of a certain colour.

Nodes have a type, a list of rules and other supporting information.

## Numerics

Work in the log domain: field gives log weights, use logsumexp to get log weight of matches, use log-max-trick to normalise match density.

## * character

A * in the rule input allows for any colour as a match, a * in the output leaves the square unchanged when applying the rule (only useful when used with a corresponding * in the input)

## Rendering:

Previously were drawing all grid squares to the screen at every update.

Change to have a background object separate from the screen, with same size. Initially draw all grid squares to the background object. At every update, draw the updated squares onto the background (drawing over old colours), then draw the background object to the screen.

This drastically reduces the number of objects drawn at each update, and so should increse performance.

## Framerate

Add clock tick to set the maximum number of renders per second. A render occurs every time a rule is successfully applied. This allows an fps setting where the display will not exceed this value.

In [542]:
class Rule():
    def __init__(self, input, output, i, j, density=None, det=False):
        # string to search for
        self.input = input
        # string to replace with
        self.output = output
        self.i_count = i
        self.j_count = j
        # deterministic / random moves
        self.det = det
        # setup field (log-weight)
        self.field = np.zeros((i, j))
        # field density function
        self.density = density

    def compute_field(self, grid):
        """Update log-probability field given a grid state."""
        if self.density:
            # compute field density
            self.field = self.density(grid)
        else:
            # default uniform field (log-weights)
            self.field = np.zeros((self.i_count, self.j_count))

    def neighbour_indices(self, i, j, n):
        """
        Return list of: lists of indices n-steps into cardinal directions about (i,j).
        """
        neighbours = []
        # search over directions
        for dx, dy in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
            steps = []
            valid = True
            for k in range(n):
                x = i + k * dx
                y = j + k * dy
                if x < 0 or y < 0 or x >= self.i_count or y >= self.j_count:
                    # out of bounds of grid
                    valid = False
                else:
                    # within bounds of grid
                    steps.append((x, y))
            # valid: no out of bounds indices
            if valid:
                neighbours.append(steps)
        return neighbours
    
    def match_distribution(self, matches):
        """
        Given a list of matches, compute their normalised distirbution
        Use log-weights of updated field
        """
        # helper function
        def logsumexp(v):
            v_max = np.max(v)
            return np.log(np.sum(np.exp(v - v_max))) + v_max
        
        # find log-weight of each match
        match_weights = []
        for match in matches:
            # log-weights of squares
            square_weights = np.array([self.field[idx] for idx in match])
            # log weight of match
            match_weights.append(logsumexp(square_weights))
        match_weights = np.array(match_weights)

        # compute normalised distribution from log-weights
        w_max = np.max(match_weights)
        dist = np.exp(match_weights - w_max) / np.sum(np.exp(match_weights - w_max))

        return dist

    def apply_rule(self, grid):
        """
        Find all matching instances of the rule input in grid,
        randomly select one to replace according to the field,
        return True, if no matches return False.
        return list of indices that were changed.
        """

        # store match indices: will be list of: lists of indices for each match
        matches = []

        # loop over grid
        for i in range(self.i_count):
            for j in range(self.j_count):
                # entry matches first letter of rule, or is *
                if (grid[i,j] == self.input[0]) or (self.input[0] == "*"):
                    # 1 letter: done
                    if len(self.input) == 1:
                        # record index
                        matches.append([(i,j)])
                    # more than 1 letter
                    else:
                        # find length
                        n = len(self.input)
                        # search n-step neighbours
                        nbs = self.neighbour_indices(i, j, n)
                        for steps in nbs:
                            # check if all n entries match rule (or rule is *)
                            matching = True
                            for k, (x, y) in enumerate(steps):
                                if not ((grid[x, y] == self.input[k]) or (self.input[k] == "*")):
                                    matching = False
                            # valid matching
                            if matching:
                                matches.append(steps)

        # no matches found: return False and no indices
        if not matches:
            return False, None
        
        # otherwise: choose a match to replace
        # update field
        self.compute_field(grid)
        # compute distribution over matches
        p = self.match_distribution(matches)
        # deterministic: choose most likely match
        if self.det:
            indices = matches[np.argmax(p)]
        # otherwise: sample from distribution
        else:
            indices = matches[np.random.choice(len(matches), p=p)]

        # apply rule to chosen match
        for k, (i, j) in enumerate(indices):
            # replace by rule: leave *'s unchanged
            if not (self.output[k] == "*"):
                grid[i, j] = self.output[k]

        # match found and replaced: return True and indices changed
        return True, indices

class Node():
    def __init__(self, type, rules, limit=None):
        # type of node
        self.type = type
        # rules in the node
        self.rules = rules
        # optional choice for limit of rule calls
        self.limit = limit

class MarkovJunior():
    def __init__(self, i, j, cellsize, fps=0):
        os.environ['SDL_VIDEO_CENTERED'] = '1'
        # keep running
        self.game = True
        # store values
        self.i_count = i
        self.j_count = j
        self.cell_size = cellsize
        # setup pygame
        pygame.init()
        # screen display
        self.screen = pygame.display.set_mode((self.j_count * self.cell_size, self.i_count * self.cell_size))
        # background object
        self.background = pygame.Surface((self.j_count * self.cell_size, self.i_count * self.cell_size))
        pygame.display.set_caption("MarkovJunior")
        self.clock = pygame.time.Clock()
        # initialise values 
        self.grid = np.empty((self.i_count, self.j_count), dtype=str)
        # colour codes
        self.ccodes = {"B": (0, 0, 0), "I": (0, 76, 153), "P": (127, 0, 255),
                       "E": (0, 153, 76), "N": (153, 76, 0), "D": (96, 96, 96),
                       "A": (192, 192, 192), "W": (255, 255, 255), "R": (255, 0, 0),
                       "O": (255, 165, 0), "Y": (255, 255, 0), "G": (0, 255, 0),
                       "U": (0, 255, 255), "S": (153, 153, 255), "K": (255, 51, 255),
                       "F": (255, 204, 153)}
        # program
        self.program = []
        # maximum framerate: default 0 has no restriction
        self.fps = fps

    def setup(self):
        """Define rules, nodes and grid."""
        # define program using nodes of rules

        # example using field density biased towards top left
        def density_top_left(grid):
            # NOTE cartesian coordinates swap axes to numpy indices
            # NOTE field should use log values
            return np.array([[-np.log(x * y + 1) for x in range(self.j_count)] for y in range(self.i_count)])

        self.program = [
            Node("Markov", [
                Rule("B", "W", self.i_count, self.j_count, density=density_top_left)
            ])
        ]

        # intitialise grid
        for i in range(self.i_count):
            for j in range(self.j_count):
                self.grid[i, j] = "B"
    
    def program_loop(self):
        """Implement program"""
        # loop over nodes in order
        for node in self.program:
            # markov node
            if node.type == "Markov":
                # loop: appling first match and resetting until no matches found
                # while rules being applied
                applied = True
                while applied:
                    applied = False
                    # loop over rules
                    for rule in node.rules:
                        # check for quit
                        self.user_input()
                        # apply each in turn
                        applied, indices = rule.apply_rule(self.grid)
                        # match found: break current loop
                        if applied:
                            # render
                            self.render(indices)
                            self.clock.tick(self.fps)
                            break
                # once no matches: end node call

            elif node.type == "Sequential":
                # loop: attempt to apply each node, until full pass with no matches
                applied = True
                while applied:
                    applied = False
                    # loop over rules
                    for rule in node.rules:
                        # check for quit
                        self.user_input()
                        # apply each in turn
                        match, indices = rule.apply_rule(self.grid)
                        # match found: turn applied true
                        if match:
                            applied = True
                            # render
                            self.render(indices)
                            self.clock.tick(self.fps)
                # once no matches in a pass: end node call

            elif node.type == "limit":
                # limit node (should have a single rule)
                # (attempt to) apply the rule 'limit' times
                for i in range(node.limit):
                    # check for quit
                    self.user_input()
                    applied, indices = node.rules.apply_rule(self.grid)
                    if applied:
                        self.render(indices)
                        self.clock.tick(self.fps)

    def render_setup(self):
        """Draw all grid squares to background object."""
        # define square function
        def square(colour, i, j):
            """Draw square at (i,j) of side length self.cellsize and colour."""
            pygame.draw.rect(self.background, colour, pygame.Rect(
                        j * self.cell_size,
                        i * self.cell_size,
                        self.cell_size,
                        self.cell_size))
        # draw all grid squares
        for i in range(self.i_count):
            for j in range(self.j_count):
                colour = self.ccodes[self.grid[i, j]]
                square(colour, i, j)


    def render(self, indices):
        """
        Draw the changed indices onto the background, colouring by entry.
        Draw the background to the screen.
        """
        # define square function
        def square(colour, i, j):
            """Draw square at (i,j) of side length self.cellsize and colour."""
            pygame.draw.rect(self.background, colour, pygame.Rect(
                        j * self.cell_size,
                        i * self.cell_size,
                        self.cell_size,
                        self.cell_size))
        # draw changes to background
        for i, j in indices:
            colour = self.ccodes[self.grid[i, j]]
            square(colour, i, j)

        # draw background to screen
        self.screen.blit(self.background, (0, 0))

        # update
        pygame.display.update()

    def user_input(self):
        """Listen for user inputs."""
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
            # listen for key presses
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    pygame.quit()

    def run(self):
        """Run main algorithm loop."""
        # call setup
        self.setup()
        # render initial grid
        self.render_setup()

        # run program
        self.program_loop()
        
        # no longer running: listen for quit
        while True:
            self.user_input()

In [543]:
pygame.quit()

In [544]:
mj = MarkovJunior(20, 20, 25, fps=100)
mj.run()

error: video system not initialized

### Chase

Create a subclass to implement a chase program using new fields and classes:

In [545]:
class Chase(MarkovJunior):
    def __init__(self, i, j, cellsize, fps=0):
        super().__init__(i, j, cellsize, fps)

    def setup(self):
        """Define program as a list of nodes of rules and populate the grid."""

        # define field density: centered about white squares
        def density_white_centre(grid):
            field = np.zeros((self.i_count, self.j_count))
            for i in range(self.i_count):
                for j in range(self.j_count):
                    # find white squares
                    if grid[i, j] == "W":
                        # compute discrete gaussian about the square
                        # NOTE: (j, i) is cartesian coordinate of numpy index [i, j]
                        def log_den(x, y):
                            pos = np.array([x, y])
                            mu = np.array([j, i])
                            sig = 1
                            return (-1 / 2*sig) * np.linalg.norm(pos - mu)**2
                        # add to field
                        field += np.array([[log_den(x, y) for x in range(self.j_count)] for y in range(self.i_count)])
                        # non-log field version:
                        # den = scipy.stats.multivariate_normal([j, i], [[1, 0], [0, 1]])
                        # field += np.array([[den.pdf([x, y]) for x in range(self.j_count)] for y in range(self.i_count)])
            return field

        # program
        self.program = [
            Node("Sequential", [
                Rule("RB", "BR", self.i_count, self.j_count, density=density_white_centre),
                Rule("WBB", "BBW", self.i_count, self.j_count)
            ])
        ]

        # intitialise grid
        for i in range(self.i_count):
            for j in range(self.j_count):
                # all black
                self.grid[i, j] = "B"
        # white point in centre
        self.grid[self.i_count // 2, self.j_count // 2] = "W"
        # origin is red
        self.grid[0, 0] = "R"

In [548]:
ch = Chase(20, 20, 20, fps=0)
ch.run()

error: display Surface quit

## Use the new * character

In [549]:
class WildcardMaze(MarkovJunior):
    def __init__(self, i, j, cellsize, fps=0):
        super().__init__(i, j, cellsize, fps)

    def setup(self):
        """Define program as a list of nodes of rules and populate the grid."""

        # program
        self.program = [
            Node("Markov", [
                Rule("RBB", "WWR", self.i_count, self.j_count),
                Rule("R*W", "W*R", self.i_count, self.j_count)
            ])
        ]

        # intitialise grid
        for i in range(self.i_count):
            for j in range(self.j_count):
                # all black
                self.grid[i, j] = "B"
        # red point in centre~
        self.grid[self.i_count // 2, self.j_count // 2] = "R"

In [550]:
wm = WildcardMaze(20, 20, 20, fps=10)
wm.run()

error: video system not initialized

## 2D rules

Create a new class of 2D rules that allow 2D inputs and outputs

In [589]:
class Rule2D(Rule):
    def __init__(self, input, output, i, j, density=None, det=False,
                 symmetry="12345678"):
        # record shape, change input and output to flat strings
        self.m, self.n = np.array([list(row) for row in input.split("/")]).shape
        input_flat = input.replace("/", "")
        output_flat = output.replace("/", "")
        # symmetries of input allowed
        self.symmetry = symmetry
        # call Rule initialiser
        super().__init__(input_flat, output_flat, i, j, density=density, det=det)

    def neighbour_indices(self, i, j):
        """
        Compute the list of indices of m x n grid with corner (i, j) for all 8 
        possible symmetries (indexing row by row with respect to original orientation)

        Return: list containing up to 8 of these index lists

        The attribute self.symmetry determines which of the 8 are returned
        - boolean list of length 8 selects exactly
        - default value selects all 8
        - give several shortcuts to allow selection of common sets:
        
        if self.symmetry is a string:
        - empty string: include only original
        - contains x: include x reflection
        - contains y: include y reflection
        - contains x and y: include xy reflection
        """

        # original
        symm_1 = [(i + p, j + q) for p in range(self.m) for q in range(self.n)]
        # reflection in y = -x
        symm_2 = [(i + p, j + q) for q in range(self.m) for p in range(self.n)]
        # 90 deg. clockwise
        symm_3 = [(i + p, j - q) for q in range(self.m) for p in range(self.n)]
        # ref. in y-axis
        symm_4 = [(i + p, j - q) for p in range(self.m) for q in range(self.n)]
        # ref. in x and y axes
        symm_5 = [(i - p, i - q) for p in range(self.m) for q in range(self.n)]
        # reflection in y = x
        symm_6 = [(i - p, i - q) for q in range(self.m) for p in range(self.n)]
        # 90 def. anti-clockwise
        symm_7 = [(i - p, j + q) for q in range(self.m) for p in range(self.n)]
        # ref. in x-axis
        symm_8 = [(i - p, j + q) for p in range(self.m) for q in range(self.n)]

        # possible options
        options = [symm_1, symm_2, symm_3, symm_4, symm_5, symm_6, symm_7, symm_8]

        # selected options
        selected = []

        # evaluate self.symmetry to decide selected
        for v in range(1, 9):
            if f"{v}" in self.symmetry:
                selected.append(options[v - 1])
                
        """
        # evaluate self.symmetry to decide selected
        if type(self.symmetry) == list:
            # boolean index
            selected = list(np.array(options)[self.symmetry])
        elif type(self.symmetry) == str:
            # original: always
            selected.append(symm_1)
            # x-ref: when x
            if "x" in self.symmetry:
                selected.append(symm_8)
            # y-ref: when y
            if "y" in self.symmetry:
                selected.append(symm_4)
            # x & y ref: when x and y
            if ("x" in self.symmetry) and ("y" in self.symmetry):
                selected.append(symm_5)
        """
        """
        # no rotation
        exact = [(p, q) for p in range(i, i + self.m) for q in range(j, j + self.n)]
        options.append(exact)
        # 90 degrees clockwise
        rotation_90 = [(p, q) for q in range(j, j - self.m, -1) for p in range(i, i + self.n)]
        if "r" in self.symmetry: options.append(rotation_90)
        # 180 degrees <=> relfection in x and y axes
        ref_xy = [(p, q) for p in range(i, i - self.m, -1) for q in range(j, j - self.n, -1)]
        if (("x" in self.symmetry) and ("y" in self.symmetry)) or ("r" in self.symmetry): options.append(ref_xy)
        # 90 degrees anticlockwise
        rotation_270 = [(p, q) for q in range(j, j + self.m) for p in range(i, i - self.n, -1)]
        if "r" in self.symmetry: options.append(rotation_270)
        # reflection in x axis
        ref_x = [(p, q) for p in range(i, i + self.m) for q in range(j, j - self.n, -1)]
        if "x" in self.symmetry: options.append(ref_x)
        # reflection in y axis
        ref_y = [(p, q) for p in range(i, i - self.m, -1) for q in range(j, j + self.n)]
        if "y" in self.symmetry: options.append(ref_y)
        """

        # valid options: indices not out of bounds
        valid_options = []

        # check if in bounds of grid
        for idxs in selected:
            valid = True
            for x, y in idxs:
                if x < 0 or y < 0 or x >= self.i_count or y >= self.j_count:
                    # out of bounds of grid
                    valid = False
            # valid: no out of bounds indices
            if valid:
                valid_options.append(idxs)
        return valid_options
    
    def match_distribution(self, matches):
        """
        Given a list of matches, compute their normalised distirbution
        Use log-weights of updated field
        """
        # helper function
        def logsumexp(v):
            v_max = np.max(v)
            return np.log(np.sum(np.exp(v - v_max))) + v_max
        
        # find log-weight of each match
        match_weights = []
        for match in matches:
            # log-weights of squares
            square_weights = np.array([self.field[idx] for idx in match])
            # log weight of match
            match_weights.append(logsumexp(square_weights))
        match_weights = np.array(match_weights)

        # compute normalised distribution from log-weights
        w_max = np.max(match_weights)
        dist = np.exp(match_weights - w_max) / np.sum(np.exp(match_weights - w_max))

        return dist

    def apply_rule(self, grid):
        """
        Find all matching instances of the rule input in grid,
        randomly select one to replace according to the field,
        return True, if no matches return False.
        return list of indices that were changed.
        """

        # store match indices: will be list of: lists of indices for each match
        matches = []

        # loop over grid
        for i in range(self.i_count):
            for j in range(self.j_count):
                # entry matches top-left letter of rule, or is *
                if (grid[i, j] == self.input[0]) or (self.input[0] == "*"):
                    # 1 letter: done
                    if len(self.input) == 1:
                        # record index
                        matches.append([(i,j)])
                    # more than 1 letter
                    else:
                        # find indices for each of the 4 rotations of rule
                        nbs = self.neighbour_indices(i, j)
                        for idx in nbs:
                            #print(f"Grid = {[grid[pos] for pos in idx]}")
                            #print(f"Rule = {[self.input[k] for k, (x, y) in enumerate(idx)]}")
                            # check if all entries match rule (or rule is *)
                            matching = True
                            for k, (x, y) in enumerate(idx):
                                if not ((grid[x, y] == self.input[k]) or (self.input[k] == "*")):
                                    matching = False
                            # valid matching
                            if matching:
                                matches.append(idx)

        # no matches found: return False and no indices
        if not matches:
            return False, None
        
        # otherwise: choose a match to replace
        # update field
        self.compute_field(grid)
        # compute distribution over matches
        p = self.match_distribution(matches)
        # deterministic: choose most likely match
        if self.det:
            indices = matches[np.argmax(p)]
        # otherwise: sample from distribution
        else:
            indices = matches[np.random.choice(len(matches), p=p)]

        # apply rule to chosen match
        for k, (i, j) in enumerate(indices):
            # replace by rule: leave *'s unchanged
            if not (self.output[k] == "*"):
                grid[i, j] = self.output[k]

        # match found and replaced: return True and indices changed
        return True, indices

In [603]:
class Fill2D(MarkovJunior):
    def __init__(self, i, j, cellsize, fps=0):
        super().__init__(i, j, cellsize, fps)

    def setup(self):
        """Define program as a list of nodes of rules and populate the grid."""

        # program
        self.program = [
            Node("Markov", [
                Rule2D("WWW/WBW", "WBW/WBW", self.i_count, self.j_count),
                Rule2D("WWW/BBB", "WBW/BBB", self.i_count, self.j_count)
            ])
        ]

        # intitialise grid
        for i in range(self.i_count):
            for j in range(self.j_count):
                if (i == 0) or (i == self.i_count - 1) or (j == 0) or (j == self.j_count - 1):
                    self.grid[i, j] = "B"
                else:
                    self.grid[i, j] = "W"

In [604]:
mj = Fill2D(30, 30, 20, fps=0)
mj.run()

error: video system not initialized

## Flowers

Implemented slightly differently to in the official markov junior: they have a set of rules for which matches are found for all, then one is chosen to be replaced. This implementatio currently can ony select from matches of a single rule.

Perhaps could be useful to create a rule subclass that allows multiple input-output strings?

## Symmetry

Ability to restrict which symmetries of input pattern are considered matches for a given rule. Currently all 4 rotations of the input are considered, but we expand this to allow selection of certain reflections using a string argument:
- empty string: only match with the exact rule input
- contains x: allow symmetry in x-axis for rules
- contins y: allow symmetry in y-axis for rules
- contains x and y: allow symmetry in both axes for rules
- contains r: allow all rotations of rules
- None: allow all

In [592]:
class Flowers(MarkovJunior):
    def __init__(self, i, j, cellsize, fps=0):
        super().__init__(i, j, cellsize, fps)

    def setup(self):
        """Define program as a list of nodes of rules and populate the grid."""

        # program
        self.program = [
            Node("limit",
                Rule2D("UUUUU/UUUUU/UUUUU/GGGGG/NNNNN", "UUUUU/UUPUU/UUEUU/GGEGG/NNENN", self.i_count, self.j_count),
                limit=1),
            Node("Sequential", [
                Rule2D("UUU/UUU/UPU", "UUU/UPU/UEU", self.i_count, self.j_count),
                Rule2D("UUU/UUU/UUU/PUU/**U", "UUU/UPU/UEU/EEU/**U", self.i_count, self.j_count, symmetry="14"),
                Rule2D("UUUUU/UUUUU/UUUUU/UUPUU/U***U", "UUUUU/UPUPU/UEUEU/UEEEU/U***U", self.i_count, self.j_count),
                Rule2D("UUU/UPU/UEU/UEU", "UYU/YEY/UYU/UEU", self.i_count, self.j_count),
                Rule2D("UUUUU/UUUUU/UUUUU/GGGGG/NNNNN", "UUUUU/UUPUU/UUEUU/GGEGG/NNENN", self.i_count, self.j_count)
            ]),
            Node("Markov", [
                Rule2D("***/*P*/***", "*Y*/YEY/*Y*", self.i_count, self.j_count)
            ])
        ]

        # intitialise grid
        for i in range(self.i_count):
            for j in range(self.j_count):
                if i == self.i_count - 4:
                    # 4th row from bottom green
                    self.grid[i, j] = "G"
                elif i > self.i_count - 4:
                    # below 4th row from bottom brown
                    self.grid[i, j] = "N"
                else:
                    # else blue
                    self.grid[i, j] = "U"

In [593]:
fl = Flowers(50, 50, 10, fps=0)
fl.run()

error: video system not initialized

In [617]:
class Testing(MarkovJunior):
    def __init__(self, i, j, cellsize, fps=0):
        super().__init__(i, j, cellsize, fps)

    def setup(self):
        """Define program as a list of nodes of rules and populate the grid."""

        # program
        self.program = [
            Node("Markov", [
                Rule2D("WB", "WW", self.i_count, self.j_count)
            ])
        ]

        # intitialise grid
        for i in range(self.i_count):
            for j in range(self.j_count):
                self.grid[i, j] = "B"
        self.grid[self.i_count // 2, self.j_count // 2] = "W"

In [618]:
test = Testing(20, 20, 20, fps=0)
test.run()

error: video system not initialized

# Restructure and clean code:

The 2D rule class works with 1D rules, so can simply put all code into a new class called Rule that allows 2D and 1D rules.

The 'apply_rule' function in the rule class is misleading and has too many tasks: finds all matches for a given rule AND attempts to apply BUT may not if there are no matches. Split into 'find_matches' function that finds all matches for a given rule and returns list of index lists for each match, and a 'replace' function that takes in a list of matches (list of index lists) and selects those to apply.

As part of the 'replace' function we add an attribute to rule class 'type' which can be "one", "all" or "prl":
- "one" rules function the same as before where a single match is chosen (sampled/deterministically) and replaced
- "all" rules "greedily choose a maximal subset of non-conflicting matches": repeatedly choose (sample/deteministically) from the distribution over matches, discarding if there are conflicts (common indices), finally replacing all chosen. [not strictly maximal guaranteed]
- "prl" rules consider all matches in parallel, replacing all regardless of conflicts (can order replacement based on sampling/det but fields and not very appropriate for parallel rules)

These 2 functions will be called by the nodes containing the rules, so there is potential for a new node type that takes multiple rules, finds matches for all of them, then calls replace on the mixed set of matches for all rules. HOWEVER, need to keep track of which rule to apply for each chosen match.

In the program we may wish to have nested nodes, perhaps mixed with rules e.g.:

Node("Sequential": \
    Node("limit", \
        Rule(...), limit = 1 \
    ) \
    Rule(...) \
)

In the MarkovJunior class we can take code out of the large program loop to simplify and allow this. Start by looping over items in self.program (should all be nodes), calling function 'run_node' on each item. 'run_node' should implement the current code, finding the node type and applying rules in the specified way HOWEVER, for each item in the rule list we now check if it is a node or a rule: if it is a rule proceed as before, if it is a node then call 'run_node' on it.
NOTE: will have to figure out how to return TRUE/FALSE from 'run_node', as need to know if any rules have been applied so that the current node can end when none have.

In [874]:
class Rule():
    def __init__(self, input, output, i, j, density=None, det=False, symmetry="12345678", type="one"):
        # record shape of rule
        self.m, self.n = np.array([list(row) for row in input.split("/")]).shape
        # string to search for
        self.input = input.replace("/", "")
        # string to replace with
        self.output = output.replace("/", "")
        # size of grid
        self.i_count = i
        self.j_count = j
        # deterministic / random moves
        self.det = det
        # setup field (log-weight)
        self.field = np.zeros((i, j))
        # field density function
        self.density = density
        # symmetries of input allowed
        self.symmetry = symmetry
        # rule type
        self.type = type

    def compute_field(self, grid):
        """Update log-probability field given a grid state."""
        if self.density:
            # compute field density
            self.field = self.density(grid)
        else:
            # default uniform field (log-weights)
            self.field = np.zeros((self.i_count, self.j_count))

    
    def neighbour_indices(self, i, j):
        """
        Compute the list of indices of m x n grid with corner (i, j) for all 8 
        possible symmetries (indexing row by row with respect to original orientation)

        Return: list containing up to 8 of these index lists

        The attribute self.symmetry determines which of the 8 are returned
        - boolean list of length 8 selects exactly
        - default value selects all 8
        - give several shortcuts to allow selection of common sets:
        
        if self.symmetry is a string:
        - empty string: include only original
        - contains x: include x reflection
        - contains y: include y reflection
        - contains x and y: include xy reflection
        """

        # original
        symm_1 = [(i + p, j + q) for p in range(self.m) for q in range(self.n)]
        # reflection in y = -x
        symm_2 = [(i + p, j + q) for q in range(self.m) for p in range(self.n)]
        # 90 deg. clockwise
        symm_3 = [(i + p, j - q) for q in range(self.m) for p in range(self.n)]
        # ref. in y-axis
        symm_4 = [(i + p, j - q) for p in range(self.m) for q in range(self.n)]
        # ref. in x and y axes
        symm_5 = [(i - p, j - q) for p in range(self.m) for q in range(self.n)]
        # reflection in y = x
        symm_6 = [(i - p, j - q) for q in range(self.m) for p in range(self.n)]
        # 90 def. anti-clockwise
        symm_7 = [(i - p, j + q) for q in range(self.m) for p in range(self.n)]
        # ref. in x-axis
        symm_8 = [(i - p, j + q) for p in range(self.m) for q in range(self.n)]

        # possible options
        options = [symm_1, symm_2, symm_3, symm_4, symm_5, symm_6, symm_7, symm_8]

        # selected options
        selected = []

        # evaluate self.symmetry to decide selected
        for v in range(1, 9):
            if f"{v}" in self.symmetry:
                selected.append(options[v - 1])

        # valid options: indices not out of bounds
        valid_options = []

        # check if in bounds of grid
        for idxs in selected:
            valid = True
            for x, y in idxs:
                if (x < 0) or (y < 0) or (x >= self.i_count) or (y >= self.j_count):
                    # out of bounds of grid
                    valid = False
            # valid: no out of bounds indices
            if valid:
                valid_options.append(idxs)
        return valid_options
    
    def match_distribution(self, matches):
        """
        Given a list of matches, compute their normalised distirbution
        Use log-weights of updated field
        """
        # helper function
        def logsumexp(v):
            v_max = np.max(v)
            return np.log(np.sum(np.exp(v - v_max))) + v_max
        
        # find log-weight of each match
        match_weights = []
        for match in matches:
            # log-weights of squares
            square_weights = np.array([self.field[idx] for idx in match])
            # log weight of match
            match_weights.append(logsumexp(square_weights))
        match_weights = np.array(match_weights)

        # compute normalised distribution from log-weights
        w_max = np.max(match_weights)
        dist = np.exp(match_weights - w_max) / np.sum(np.exp(match_weights - w_max))

        return dist
    
    def find_matches(self, grid):
        """
        Search for all matching instances of the rule input in the grid
        Return: list of index lists for each match
        """

        # store match indices: will be list of: lists of indices for each match
        matches = []

        # loop over grid
        for i in range(self.i_count):
            for j in range(self.j_count):
                # entry matches top-left letter of rule, or is *
                if (grid[i, j] == self.input[0]) or (self.input[0] == "*"):
                    # 1 letter: done
                    if len(self.input) == 1:
                        # record index
                        matches.append([(i,j)])
                    # more than 1 letter
                    else:
                        # find indices for each of the 4 rotations of rule
                        nbs = self.neighbour_indices(i, j)
                        for idx in nbs:
                            #print(f"Grid = {[grid[pos] for pos in idx]}")
                            #print(f"Rule = {[self.input[k] for k, (x, y) in enumerate(idx)]}")
                            # check if all entries match rule (or rule is *)
                            matching = True
                            for k, (x, y) in enumerate(idx):
                                if not ((grid[x, y] == self.input[k]) or (self.input[k] == "*")):
                                    matching = False
                            # valid matching
                            if matching:
                                matches.append(idx)
        #print(len(matches))
        return matches
    
    def replace(self, matches, grid):
        """
        Compute distribution over given list of matches, sample or choose
        deterministically matches, replace those chosen.

        Type of rule:
        - "one": choose a single match
        - "all": choose "maximal" set of non-conflicting matches
        - "prl": choose all matches, regardless of overlap
        """
        
        # compute field
        self.compute_field(grid)
        # compute distribution over matches
        p = self.match_distribution(matches)

        # indices of chosen matches
        indices = []

        # one type
        if self.type == "one":
            # deterministic: choose most likely match
            if self.det:
                indices.append(matches[np.argmax(p)])
            # otherwise: sample from distribution
            else:
                k = np.random.choice(len(matches), p=p)
                indices.append(matches[k])

        # all type
        elif self.type == "all":
            while matches:
                # re-normalise distribution (do at start otherwise last loop gives error)
                p = p / np.sum(p)
                # choose from remaining matches
                # deterministic
                if self.det:
                    k = np.argmax(p)
                # not: sample
                else:
                    k = np.random.choice(len(matches), p=p)
                # check for conflicts
                valid = True
                for chosen in indices:
                    # common square between prev. chosen and new sample
                    if set(chosen) & set(matches[k]):
                        # discard
                        valid = False
                # valid match
                if valid:
                    # store chosen match
                    indices.append(matches[k])
                # remove match (regardless of validity)
                matches.pop(k)
                p = np.delete(p, k)

        # parallel type
        elif self.type == "prl":
            # indices of matches, sorted order of increasing prob.
            idxs = sorted(range(len(p)), key=lambda k: p[k])
            # choose all matches in this order (so that higher prob overlap last)
            for k in idxs:
                indices.append(matches[k])
            
        # loop over all chosen matches
        for index in indices:
            # apply rule to each
            for k, (i, j) in enumerate(index):
                # replace by rule: leave *'s unchanged
                if not (self.output[k] == "*"):
                    grid[i, j] = self.output[k]

        # return changed indices
        return indices

In [875]:
class Node():
    def __init__(self, type, rules, limit=None):
        # type of node
        self.type = type
        # rules in the node
        self.rules = rules
        # optional choice for limit of rule calls
        self.limit = limit

In [876]:
class MarkovJunior():
    def __init__(self, i, j, cellsize, fps=0):
        os.environ['SDL_VIDEO_CENTERED'] = '1'
        # keep running
        self.game = True
        # store values
        self.i_count = i
        self.j_count = j
        self.cell_size = cellsize
        # setup pygame
        pygame.init()
        # screen display
        self.screen = pygame.display.set_mode((self.j_count * self.cell_size, self.i_count * self.cell_size))
        # background object
        self.background = pygame.Surface((self.j_count * self.cell_size, self.i_count * self.cell_size))
        pygame.display.set_caption("MarkovJunior")
        self.clock = pygame.time.Clock()
        # initialise values 
        self.grid = np.empty((self.i_count, self.j_count), dtype=str)
        # colour codes
        self.ccodes = {"B": (0, 0, 0), "I": (0, 76, 153), "P": (127, 0, 255),
                       "E": (0, 153, 76), "N": (153, 76, 0), "D": (96, 96, 96),
                       "A": (192, 192, 192), "W": (255, 255, 255), "R": (255, 0, 0),
                       "O": (255, 165, 0), "Y": (255, 255, 0), "G": (0, 255, 0),
                       "U": (0, 255, 255), "S": (153, 153, 255), "K": (255, 51, 255),
                       "F": (255, 204, 153)}
        # program
        self.program = []
        # maximum framerate: default 0 has no restriction
        self.fps = fps

    def setup(self):
        """Define rules, nodes and grid."""
        # define program using nodes of rules

        # example using field density biased towards top left
        def density_top_left(grid):
            # NOTE cartesian coordinates swap axes to numpy indices
            # NOTE field should use log values
            return np.array([[-np.log(x * y + 1) for x in range(self.j_count)] for y in range(self.i_count)])

        self.program = [
            Node("Markov", [
                Rule("B", "W", self.i_count, self.j_count, density=density_top_left)
            ])
        ]

        # intitialise grid
        for i in range(self.i_count):
            for j in range(self.j_count):
                self.grid[i, j] = "B"

    def run_node(self, node):
        # markov node
        if node.type == "Markov":
            # loop: applying first match and resetting until no matches found
            # while rules being applied
            applied = True
            while applied:
                applied = False
                # loop over rules
                for rule in node.rules:
                    # check for quit
                    self.user_input()
                    # find matches
                    matches = rule.find_matches(self.grid)
                    # match found (non-empty list): break current loop
                    if matches:
                        applied = True
                        # replace matches: get list of changes
                        indices = rule.replace(matches, self.grid)
                        # render changes
                        self.render(indices)
                        self.clock.tick(self.fps)
                        break
            # once no matches: end node call

        elif node.type == "Sequential":
            # loop: attempt to apply each node, until full pass with no matches
            applied = True
            while applied:
                applied = False
                # loop over rules
                for rule in node.rules:
                    # check for quit
                    self.user_input()
                    # find matches
                    matches = rule.find_matches(self.grid)
                    # match found (non-empty list): turn applied true
                    if matches:
                        applied = True
                        # replace matches: get list of changes
                        indices = rule.replace(matches, self.grid)
                        # render changes
                        self.render(indices)
                        self.clock.tick(self.fps)
            # once no matches in a pass: end node call

        elif node.type == "limit":
            # limit node (should have a single rule)
            # (attempt to) apply the rule 'limit' times
            for i in range(node.limit):
                # check for quit
                self.user_input()
                # find matches
                matches = node.rules.find_matches(self.grid)
                # match found (non-empty list) 
                if matches:
                    # replace matches: get list of changes
                    indices = node.rules.replace(matches, self.grid)
                    # render changes
                    self.render(indices)
                    self.clock.tick(self.fps)

    
    def program_loop(self):
        """Implement program"""
        # loop over nodes in order
        for node in self.program:
            # run node
            self.run_node(node)

    def render_setup(self):
        """Draw all grid squares to background object."""
        # define square function
        def square(colour, i, j):
            """Draw square at (i,j) of side length self.cellsize and colour."""
            pygame.draw.rect(self.background, colour, pygame.Rect(
                        j * self.cell_size,
                        i * self.cell_size,
                        self.cell_size,
                        self.cell_size))
        # draw all grid squares
        for i in range(self.i_count):
            for j in range(self.j_count):
                colour = self.ccodes[self.grid[i, j]]
                square(colour, i, j)


    def render(self, indices):
        """
        Draw the changed indices onto the background, colouring by entry.
        Draw the background to the screen.
        """
        # define square function
        def square(colour, i, j):
            """Draw square at (i,j) of side length self.cellsize and colour."""
            pygame.draw.rect(self.background, colour, pygame.Rect(
                        j * self.cell_size,
                        i * self.cell_size,
                        self.cell_size,
                        self.cell_size))
        # draw changes to background: 
        # indices is a list of index lists, one for each match
        for index in indices:
            for i, j in index:
                colour = self.ccodes[self.grid[i, j]]
                square(colour, i, j)

        # draw background to screen
        self.screen.blit(self.background, (0, 0))

        # update
        pygame.display.update()

    def user_input(self):
        """Listen for user inputs."""
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
            # listen for key presses
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    pygame.quit()

    def run(self):
        """Run main algorithm loop."""
        # call setup
        self.setup()
        # render initial grid
        self.render_setup()

        # run program
        self.program_loop()
        
        # no longer running: listen for quit
        while True:
            self.user_input()

## Test

In [877]:
class Testing(MarkovJunior):
    def __init__(self, i, j, cellsize, fps=0):
        super().__init__(i, j, cellsize, fps)

    def setup(self):
        """Define program as a list of nodes of rules and populate the grid."""

        # program
        self.program = [
            Node("Markov", [
                Rule("WB", "WW", self.i_count, self.j_count, type="all")
            ])
        ]

        # intitialise grid
        for i in range(self.i_count):
            for j in range(self.j_count):
                self.grid[i, j] = "B"
        self.grid[self.i_count // 2, self.j_count // 2] = "W"

In [878]:
test = Testing(100, 100, 5, fps=0)
test.run()

error: video system not initialized

## Knight moves

In [892]:
class Knight(MarkovJunior):
    def __init__(self, i, j, cellsize, fps=0):
        super().__init__(i, j, cellsize, fps)

    def setup(self):
        """Define program as a list of nodes of rules and populate the grid."""

        # program
        self.program = [
            Node("limit",
                 Rule("A", "W", self.i_count, self.j_count),
                 limit=1),
            Node("limit",
                 Rule("A", "B", self.i_count, self.j_count),
                 limit=1),
            Node("Sequential", [
                Rule("W**/***", "A**/**W", self.i_count, self.j_count, symmetry="12345678", type="one"),
                Rule("B**/***", "A**/**B", self.i_count, self.j_count, symmetry="12345678", type="one")
            ])
        ]

        # intitialise grid
        for i in range(self.i_count):
            for j in range(self.j_count):
                self.grid[i, j] = "A"

In [894]:
kn = Knight(20, 20, 20, fps=30)
kn.run()

error: display Surface quit

# Nested Growth

In [904]:
class NestedGrowth(MarkovJunior):
    def __init__(self, i, j, cellsize, fps=0):
        super().__init__(i, j, cellsize, fps)

    def setup(self):
        """Define program as a list of nodes of rules and populate the grid."""

        def star(a, b):
            return f"*{a}{a}{a}*/{a}{a}{a}{a}{a}/{a}{a}{b}{a}{a}/{a}{a}{a}{a}{a}/*{a}{a}{a}*"

        # program
        self.program = [
            Node("Sequential", [
                Rule("WB", "WW", self.i_count, self.j_count, type="all"),
                Rule("AW", "AA", self.i_count, self.j_count, type="all"),
                Rule("DA", "DD", self.i_count, self.j_count, type="all"),
                Rule("WD", "WW", self.i_count, self.j_count, type="all"),
                Rule(star("W","W"), star("W","A"), self.i_count, self.j_count, type="all"),
                Rule(star("A","A"), star("A","D"), self.i_count, self.j_count, type="all"),
                Rule(star("D","D"), star("D","W"), self.i_count, self.j_count, type="all")
            ])
        ]

        # intitialise grid
        for i in range(self.i_count):
            for j in range(self.j_count):
                self.grid[i, j] = "B"
        self.grid[self.i_count // 2, self.j_count // 2] = "W"

In [906]:
ng = NestedGrowth(10, 10, 20, fps=0)
ng.run()

error: video system not initialized